In [1]:
import time
import requests
import json
from authlib.jose import JsonWebSignature
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()  # Carga las variables del archivo .env

token_id = os.getenv("API_TOKEN_ID")
token_secret = os.getenv("API_TOKEN_SECRET")

In [ ]:
request = {
    "method": "GET",
    "contentType": "application/json",
    "uri": "/v3/reporting/devices",
    # The queryString is URL encoded
    "queryString": "pageSize=200",
    #"queryString": "orderBy=esn%2CdeviceName%3Adesc%2CdeviceUid%3Aasc",
    # For GET requests, the payload is empty
    "payload": {}
}

In [18]:
request_payload_data = {
    "data": request["payload"]
}
headers = {
    "alg": "HS256",
    "kid": token_id,
    "method": request["method"],
    "content-type": request["contentType"],
    "uri": request["uri"],
    "query-string": request["queryString"],
    "issuedAt": round(time.time() * 1000)
}

jws = JsonWebSignature()
signed = jws.serialize_compact(headers, json.dumps(request_payload_data), token_secret)


In [19]:
# If you log in to https://cc.absolute.com,
# use https://api.absolute.com/jws/validate.

request_url = "https://api.absolute.com/jws/validate"
r = requests.post(request_url, signed, {"content-type": "text/plain"})

In [20]:
data = r.json()
df = pd.DataFrame([data])

In [22]:
data = r.json()
df = pd.DataFrame([data])

df= df.explode('data',ignore_index=True)
df_dispositivos = pd.concat([df.drop(columns=["data"]), df["data"].apply(pd.Series)], axis=1)

In [23]:
df.head(3)

,data,metadata
0,{'deviceUid': '082bbc5e-506a-4688-82bc-e667aa1...,{'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...
1,{'deviceUid': '109bdcbd-4147-4650-8753-a42ad2c...,{'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...
2,{'deviceUid': '1ad09ff3-8a3f-47d0-bac1-022570b...,{'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...


In [28]:
print(df.iloc[2])

data        {'deviceUid': '1ad09ff3-8a3f-47d0-bac1-022570b...
metadata    {'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...
Name: 2, dtype: object


In [11]:
df_dispositivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   metadata                200 non-null    object
 1   deviceUid               200 non-null    object
 2   accountUid              200 non-null    object
 3   appName                 200 non-null    object
 4   esn                     200 non-null    object
 5   deviceName              200 non-null    object
 6   deviceSerialNumber      200 non-null    object
 7   userName                200 non-null    object
 8   installPath             200 non-null    object
 9   installDateTimeUtc      200 non-null    object
 10  osName                  200 non-null    object
 11  major                   200 non-null    int64 
 12  minor                   200 non-null    int64 
 13  deviceAppId             200 non-null    object
 14  appId                   200 non-null    object
 15  appOri

In [25]:
# Configuracion para que pandas muestre todas las columnas
pd.set_option("display.max_columns", None)

# Configuración para mostrar números en notación normal
pd.set_option("display.float_format", "{:.0f}".format)

df_dispositivos.head(3)

,metadata,deviceUid,accountUid,appName,esn,deviceName,deviceSerialNumber,userName,installPath,installDateTimeUtc,osName,major,minor,deviceAppId,appId,appOriginalName,appPublisher,appOriginalPublisher,appVersion,appOriginalVersion,firstDetectDateTimeUtc,lastScanDateTimeUtc
0,{'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...,082bbc5e-506a-4688-82bc-e667aa17f3ac,fc39ea8e-61df-4330-aaca-a6ac74f0b025,TightVNC,6LJCEBX5M9AA4IEM0016,NB-CEM038,5CD42275ML,ARUNTNET\terrencio.cruz,c:\program files\tightvnc,2024-08-02T05:00:00Z,Microsoft Windows 11 Pro,2,8,40cfcbad-3d1a-35de-85c4-13a951d556a0,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,TightVNC,GlavSoft,GlavSoft LLC.,2.8.81.0,2.8.81.0,2024-08-02T21:58:55.591Z,2024-12-12T15:28:56.624Z
1,{'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...,109bdcbd-4147-4650-8753-a42ad2c6b691,fc39ea8e-61df-4330-aaca-a6ac74f0b025,TightVNC,6LJCEBX5M9AA4IEM0057,NB-CEM061,5CG4311L6S,ARUNTNET\toninho.serrano,c:\program files\tightvnc,2024-12-06T05:00:00Z,Microsoft Windows 11 Pro,2,8,5d45297c-9cb6-38e8-b773-95b49bc8cb2c,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,TightVNC,GlavSoft,GlavSoft LLC.,2.8.81.0,2.8.81.0,2024-12-07T00:23:34.615Z,2024-12-12T20:14:07.969Z
2,{'pagination': {'nextPage': 'WyIwMzQ4MzhjZi04M...,1ad09ff3-8a3f-47d0-bac1-022570b6a50c,fc39ea8e-61df-4330-aaca-a6ac74f0b025,TightVNC,6LJCEBX5M9AA4I350057,NB-CEM022,4CE336D54Z,CEMPROTECH\ysalas,c:\program files\tightvnc,2024-08-02T05:00:00Z,Microsoft Windows 11 Pro,2,8,e2991a0b-d1ac-3385-863f-d9455b5f4090,0005f8a9-26f2-3f3a-99b7-5ff6fd863459,TightVNC,GlavSoft,GlavSoft LLC.,2.8.81.0,2.8.81.0,2024-10-01T18:52:58.354Z,2024-12-12T14:46:48.615Z


In [178]:
df_dispositivos= df_dispositivos[['deviceUid','serialNumber', 'systemManufacturer', 'systemModel', 
                                  'username', 'lastUpdatedDateTimeUtc', 'platformOSType', 'deviceName', 
                                  'totalPhysicalRamBytes', 'availablePhysicalRamBytes', 'domain', 'localIp',
                                  'cpu', 'bios', 'avpInfo', 'geoData']]

In [179]:
df_dispositivos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   deviceUid                  117 non-null    object 
 1   serialNumber               116 non-null    object 
 2   systemManufacturer         116 non-null    object 
 3   systemModel                116 non-null    object 
 4   username                   116 non-null    object 
 5   lastUpdatedDateTimeUtc     117 non-null    object 
 6   platformOSType             117 non-null    object 
 7   deviceName                 116 non-null    object 
 8   totalPhysicalRamBytes      116 non-null    float64
 9   availablePhysicalRamBytes  116 non-null    float64
 10  domain                     116 non-null    object 
 11  localIp                    116 non-null    object 
 12  cpu                        116 non-null    object 
 13  bios                       116 non-null    object 

In [180]:
df_dispositivos["cpu_name"] = df_dispositivos["cpu"].apply(lambda x: x["name"] if isinstance(x, dict) else None)
df_dispositivos["bios_ver"] = df_dispositivos["bios"].apply(lambda x: x["versionDate"] if isinstance(x, dict) else None)
df_dispositivos["latitude"] = df_dispositivos["geoData"].apply(
                                lambda x: x["location"]["point"]["y"] if isinstance(x, dict) else None)

df_dispositivos["longitude"] = df_dispositivos["geoData"].apply(
                                lambda x: x["location"]["point"]["x"] if isinstance(x, dict) else None)

df_dispositivos["antivirus"] = df_dispositivos["avpInfo"].apply(
                                lambda x: f"{x.get('antivirusName', 'Unknown')} {x.get('antivirusVersion', 'Unknown')}" 
                                if isinstance(x, dict) else None)

df_dispositivos['MemoriaRamTotal']= (df_dispositivos['totalPhysicalRamBytes'] / 1024) / 1024
df_dispositivos['MemoriaRamDisponible']= (df_dispositivos['availablePhysicalRamBytes'] / 1024) / 1024

In [181]:
# Eliminar columnas innecesarias
df_dispositivos= df_dispositivos.drop(['cpu','bios', 'geoData', 'avpInfo', 'totalPhysicalRamBytes', 'availablePhysicalRamBytes'] , axis=1) 

# cambiar normbre de la columna
df_dispositivos= df_dispositivos.rename(columns={'cpu_name': 'cpu', 'bios_ver': 'bios'}) 


In [182]:
df_dispositivos.head(5)

,deviceUid,serialNumber,systemManufacturer,systemModel,username,lastUpdatedDateTimeUtc,platformOSType,deviceName,domain,localIp,cpu,bios,latitude,longitude,antivirus,MemoriaRamTotal,MemoriaRamDisponible
0,f81ffad4-31cf-4504-8fe3-ecf2d4559be0,NaN,NaN,NaN,NaN,2024-05-20T18:55:33.212Z,Unknown,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN
1,4c498db0-3ce3-4fd0-962c-4403474fe6ab,PF4Q3GBK,Lenovo,THINKPAD T16 GEN 2,ARUNTNET\fruiz,2024-06-10T13:43:32.745Z,Windows,LAP-QOR-CRP-03,aruntnet.local,172.16.21.140,13th Gen Intel(R) Core(TM) i7-1355U,"LENOVO N3QET44W (1.44 ), 03/13/2024",-12,-77,Sophos Intercept X 3.2.0.1560,16057,3111
2,b0142679-a879-4808-850a-66fcb61fe825,4CE336D4QK,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\monica.ulloa,2024-12-02T13:34:06.382Z,Windows,PC-QOR-RENT-03,aruntnet.local,172.16.65.18,12th Gen Intel(R) Core(TM) i7-12700,"HP U03 Ver. 02.15.20, 09/12/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,4653
3,6a5a0186-d33e-4441-b673-7746a9d6a05b,4CE336D5BV,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\karen.valenzuela,2024-12-02T13:34:06.284Z,Windows,PC-QOR-RENT-02,aruntnet.local,172.16.65.56,12th Gen Intel(R) Core(TM) i7-12700,"HP U03 Ver. 02.15.20, 09/12/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,408
4,e0c383a0-570b-49b2-8816-81a898e03b84,4CE336D556,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\rtafur,2024-12-02T13:22:17.000Z,Windows,PC-QOR-RENT-01,aruntnet.local,172.16.64.135,12th Gen Intel(R) Core(TM) i7-12700,"HP U03 Ver. 02.15.20, 09/12/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,5305


In [183]:
df_dispositivos

,deviceUid,serialNumber,systemManufacturer,systemModel,username,lastUpdatedDateTimeUtc,platformOSType,deviceName,domain,localIp,cpu,bios,latitude,longitude,antivirus,MemoriaRamTotal,MemoriaRamDisponible
0,f81ffad4-31cf-4504-8fe3-ecf2d4559be0,NaN,NaN,NaN,NaN,2024-05-20T18:55:33.212Z,Unknown,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,NaN
1,4c498db0-3ce3-4fd0-962c-4403474fe6ab,PF4Q3GBK,Lenovo,THINKPAD T16 GEN 2,ARUNTNET\fruiz,2024-06-10T13:43:32.745Z,Windows,LAP-QOR-CRP-03,aruntnet.local,172.16.21.140,13th Gen Intel(R) Core(TM) i7-1355U,"LENOVO N3QET44W (1.44 ), 03/13/2024",-12,-77,Sophos Intercept X 3.2.0.1560,16057,3111
2,b0142679-a879-4808-850a-66fcb61fe825,4CE336D4QK,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\monica.ulloa,2024-12-02T13:34:06.382Z,Windows,PC-QOR-RENT-03,aruntnet.local,172.16.65.18,12th Gen Intel(R) Core(TM) i7-12700,"HP U03 Ver. 02.15.20, 09/12/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,4653
3,6a5a0186-d33e-4441-b673-7746a9d6a05b,4CE336D5BV,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\karen.valenzuela,2024-12-02T13:34:06.284Z,Windows,PC-QOR-RENT-02,aruntnet.local,172.16.65.56,12th Gen Intel(R) Core(TM) i7-12700,"HP U03 Ver. 02.15.20, 09/12/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,408
4,e0c383a0-570b-49b2-8816-81a898e03b84,4CE336D556,Hewlett-Packard,HP PRO SFF 400 G9 DESKTOP PC,ARUNTNET\rtafur,2024-12-02T13:22:17.000Z,Windows,PC-QOR-RENT-01,aruntnet.local,172.16.64.135,12th Gen Intel(R) Core(TM) i7-12700,"HP U03 Ver. 02.15.20, 09/12/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,16054,5305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,e37bcbd9-4ec7-4d8c-97f8-b62c3746bf42,5CD42275MK,Hewlett-Packard,HP ZBOOK POWER 16 INCH G11 MOBILE WORKSTATION PC,ARUNTNET\wnajera,2024-11-22T14:58:30.546Z,Windows,LAP-QOR-RENT-20,aruntnet.local,172.16.66.240,Intel(R) Core(TM) Ultra 9 185H,"HP W97 Ver. 01.01.05, 04/22/2024",NaN,NaN,Sophos Intercept X 3.3.1.2,32188,21756
113,e56d7683-c206-4515-ae8f-f28a97cb0342,5CD42275MD,Hewlett-Packard,HP ZBOOK POWER 16 INCH G11 MOBILE WORKSTATION PC,ARUNTNET\emonzon,2024-11-27T15:04:08.651Z,Windows,LAP-QOR-RENT-21,aruntnet.local,172.16.66.113,Intel(R) Core(TM) Ultra 9 185H,"HP W97 Ver. 01.03.04, 09/06/2024",-12,-77,Sophos Endpoint Agent 2024.2.4.1.0,32188,21026
114,9d5eec55-a13d-4ad4-814e-5b70c9cf601b,5CD410J5V0,Hewlett-Packard,HP PROBOOK 450 15.6 INCH G10 NOTEBOOK PC,ARUNTNET\magaly.bardales,2024-12-02T13:13:05.516Z,Windows,CRESLAPMA03,aruntnet.local,172.16.32.228,13th Gen Intel(R) Core(TM) i7-1355U,"HP V72 Ver. 01.06.03, 08/15/2024",-15,-72,Sophos Intercept X 3.3.1.2,16016,6443
115,55002370-df66-4d87-9576-4e7413525027,5CD410J5V1,Hewlett-Packard,HP PROBOOK 450 15.6 INCH G10 NOTEBOOK PC,ARUNTNET\mvela,2024-11-22T00:10:30.410Z,Windows,CRESLAPMA01,aruntnet.local,192.168.20.211,13th Gen Intel(R) Core(TM) i7-1355U,"HP V72 Ver. 01.05.05, 06/17/2024",-15,-72,Sophos Intercept X 3.3.1.2,16016,9081


In [184]:
print(df_dispositivos.iloc[2])
#print(df_dispositivos.iloc[1]['avpInfo'])

deviceUid                 b0142679-a879-4808-850a-66fcb61fe825
serialNumber                                        4CE336D4QK
systemManufacturer                             Hewlett-Packard
systemModel                       HP PRO SFF 400 G9 DESKTOP PC
username                                 ARUNTNET\monica.ulloa
lastUpdatedDateTimeUtc                2024-12-02T13:34:06.382Z
platformOSType                                         Windows
deviceName                                      PC-QOR-RENT-03
domain                                          aruntnet.local
localIp                                           172.16.65.18
cpu                        12th Gen Intel(R) Core(TM) i7-12700
bios                          HP U03 Ver. 02.15.20, 09/12/2024
latitude                                                   -12
longitude                                                  -77
antivirus                   Sophos Endpoint Agent 2024.2.4.1.0
MemoriaRamTotal                                        